# Make a plot of the data

# Make a plot of the data

# Première expérience
Nous allons si nous arrivons à faire un autoencoder qui fonctionne correctement (100% accuracy) 
Cette auto encodeur servira à fournir l'espace lattent au modèle prédictif de coups d'échecs

In [22]:
import torch

# Detection du gpu et utilisation si disponible
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda:0


In [23]:
from pathlib import Path

# Hyper paramètres
lr = 1e-3
batch_size = 8
num_epochs = 200

# Chemin vers le fichier de sauvegarde des poids
path = Path().resolve().parent / 'weights' / f'auto_encoder_64.pth'
print(path)

C:\Users\pierr\OneDrive\Bureau\Projet\chess_pytorch_ia\weights\auto_encoder_64.pth


In [24]:
from chess_pytorch_ia.autoencoder.model_64 import AutoEncoder64
from torch import nn

# Charge l'architecture de l'auto encodeur 64x (8x8)
model = AutoEncoder64().to(device=device)

In [25]:
from torch.utils.data import DataLoader
from chess_pytorch_ia.autoencoder.datasets.dataset_64 import RandomMoveDataset64

# Charge les données
dataset = RandomMoveDataset64()
loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

In [26]:
# Fonction pour calculer la précision du modèle
from chess_pytorch_ia.analys import check_accuracy

In [27]:
from torch import optim

# Fonction de perte et optimiseur
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [28]:
from IPython.display import display, clear_output
from tqdm.notebook import tqdm

# Entraînement du modèle et sauvegarde du meilleur modèle
best_accuracy = 0
for epoch in range(num_epochs):
    pbar = tqdm(enumerate(loader), total=len(loader), leave=False)

    for batch_idx, (data, targets) in pbar:
        data = data.to(device=device)
        targets = torch.argmax(targets, dim=1)
        targets = targets.to(device=device)

        scores = model.forward(data)
        _, predictions = scores.max(1)

        accuracy = (predictions == targets).sum()
        accuracy = f"{(float(accuracy) / batch_size) * 100:.2f}"

        loss = criterion(scores, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pbar.set_description(f"epochs: {epoch + 1}/{num_epochs}  -  accuracy : {accuracy}  -  loss: {loss.item():.4f}")

    accuracy = check_accuracy(loader, model, device=device)
    pbar.refresh()

    # Utilisation de display et clear_output pour contrôler l'affichage
    clear_output(wait=True)
    display(f"Epoch: {epoch + 1}/{num_epochs} - Training Loss: {loss.item():.4f} - Accuracy: {accuracy}")

    if float(best_accuracy) < float(accuracy):
        best_accuracy = accuracy
        print("Ce modèle est meilleur que le précédent, sauvegarde en cours..")
        torch.save(model.state_dict(), f'{path}')

print(f"Le meilleur modèle a {best_accuracy}% de précision")

'Epoch: 200/200 - Training Loss: 3.1887 - Accuracy: 100.00'

Le meilleur modèle a 100.00% de précision


In [ ]:
# Entrainement du modèle de prediction de coups

# Hyper paramètres
lr = 1e-3
batch_size = 8
num_epochs = 200

# Chemin vers le fichier de sauvegarde des poids
path = Path().resolve().parent / 'weights' / f'auto_encoder_64.pth'
print(path)

In [ ]:

# Detection du gpu et utilisation si disponible
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:

from chess_pytorch_ia.autoencoder.model_64 import AutoEncoder64
from torch import nn
    
# Charge l'architecture de l'auto encodeur 64x (8x8)
model = AutoEncoder64().to(device=device)

In [ ]:

from torch.utils.data import DataLoader
from chess_pytorch_ia.autoencoder.datasets.dataset_64 import RandomMoveDataset64